# Setup Notebook

Download environmental variables for Java and Google Cloud application key

In [4]:
%env GOOGLE_APPLICATION_CREDENTIALS=/root/Documents/data/NDEX-e06611c0a4d0.json
%env JAVA_HOME=/usr/lib/jvm/java-8-openjdk-amd64

env: GOOGLE_APPLICATION_CREDENTIALS=/root/Documents/data/NDEX-e06611c0a4d0.json
env: JAVA_HOME=/usr/lib/jvm/java-8-openjdk-amd64


In [5]:
import redis
import os

In [6]:
from google.cloud import datastore

In [7]:
import sys
import numpy as np
from pprint import pprint
import inspect

In [7]:
sys.path.append("/root/Documents/indra")
from indra.sources import biopax
from indra.sources import trips


In [4]:
#from indra import assemblers
from indra.assemblers  import cx_assembler
import ndex

ImportError: No module named 'indra.assemblers'

In [39]:
from IPython.core.display import HTML, display
# to reverse this feature restart kernel with "restart and clear output"
display(HTML("<style> *{margin:0; padding:0;} html, body, \
             .container{margin:0;!important padding:0;!important} \
             .container { width:100% !important;}</style>"))

# Check that Packages Work
We check that all the essential packages work. Below we explain essential packages:
* **Indra : ** used from scraping data from files
* **Datastore : ** the main storage database on google's cloud
* **Redis : ** an ultra-fast RAM database on google
* **ndb Model** a common format for writing Datastore models <font color="gray">(optional)</font>

## Check Indra

In [25]:
biopax_processor = biopax.biopax_api.process_owl("data/EGFR-dependent-Endothelin-signaling-events.owl")
biopax_processor.get_complexes()

statements = biopax_processor.statements
cx_test = cx_assembler.CxAssembler(stmts=statements, network_name="large_network2")

cx_test.make_model(add_indra_json=True)
cx_test.save_model(file_name='model.cx')

In [26]:
statements

[ActiveForm(MTOR(mods: (phosphorylation, S, 2448)), activity, True),
 Complex(EGF(), EGFR()),
 Complex(GRB2(), SOS1()),
 Complex(GTP(), HRAS()),
 Complex(EDN1(), EDNRA()),
 Complex(GDP(), HRAS()),
 Complex(EGF(), EGF(), EGFR(mods: (phosphorylation, Y)), EGFR(mods: (phosphorylation, Y)), SHC1(mods: (phosphorylation, Y, 240), (phosphorylation, Y, 239), (phosphorylation, Y, 317))),
 Complex(EGF(), EGF(), EGFR(mods: (phosphorylation, Y)), EGFR(mods: (phosphorylation, Y))),
 Complex(GRB2(), EGF(), EGF(), SOS1(), EGFR(mods: (phosphorylation, Y)), EGFR(mods: (phosphorylation, Y)), SHC1(mods: (phosphorylation, Y, 240), (phosphorylation, Y, 239), (phosphorylation, Y, 317)))]

## Check Datastore

In [8]:
client = datastore.Client()

In [9]:
# The kind for the new entity
kind = 'Task'
# The name/ID for the new entity
name = 'sampletask1'
# The Cloud Datastore key for the new entity
task_key = client.key(kind, name)

In [10]:
task = datastore.Entity(key=task_key)
task['description'] = 'Buy milk'

In [11]:
task

<Entity('Task', 'sampletask1') {'description': 'Buy milk'}>

In [13]:
client.put(task)

## Check Redis

In [41]:
#r = redis.Redis(
#    host='hostname',
#    port=port, 
#    password='password')
r_client = redis.StrictRedis(host='10.0.0.3', port=6379)

In [42]:
value = r_client.get('HELLO')
value

b'WORLD'

# Simple Upload Example
We take a [.CX](http://35.232.154.170/edit/small.cx) file created by Indra and upload it to [google cloud](https://cloud.google.com) 

## Read JSON
We read the JSON file into an adjacency list

In [26]:
with open("small.cx", "r") as read_file:
    data = json.load(read_file)

In [27]:
nodes = data[4]['nodes']
edges = data[5]['edges']

In [28]:
db={}
for node in nodes:
    db[ str(node['@id']) ] =[]

for edge in edges:
    source, target = str(edge['s']), edge['t']
    if target not in db[source]: 
        db[source].append( target )

## Upload to Datastore
We upload the adjacency list to datastore

In [33]:
client = datastore.Client()

AttributeError: module 'google.datastore' has no attribute 'Client'

In [103]:
for key,val in db.items() :
    db_key = client.key("protein", key)
    entity = datastore.Entity(key=db_key)
    entity['edges'] = val
    print(entity)
    client.put(entity)

<Entity('protein', '10') {'edges': []}>
<Entity('protein', '0') {'edges': ['1', '0', '17', '4']}>
<Entity('protein', '1') {'edges': ['1', '17']}>
<Entity('protein', '4') {'edges': ['1', '17']}>
<Entity('protein', '9') {'edges': ['10']}>
<Entity('protein', '3') {'edges': ['4', '0', '1', '17']}>
<Entity('protein', '17') {'edges': []}>
<Entity('protein', '6') {'edges': ['7']}>
<Entity('protein', '12') {'edges': ['7']}>
<Entity('protein', '7') {'edges': []}>


## Upload to Redis
Save adjacency list into [Redis](https://redis.io/) (a <font color='magenta'>RAM database</font>)

In [104]:
import redis
r_client = redis.StrictRedis(host='10.0.0.3', port=6379)

In [118]:
for key,val in db.items():
    for target in val :
        r_client.rpush(key, target)

In [121]:
r_client.lrange('0', 0, -1)

[b'1', b'0', b'17', b'4']